Start Time == 9:01 PM

Run Time == ~2 Minutes

Drive Code Steps:

- GLOB FOR YESTERDAYS AND TODAYS FILE(S)
- CREATE DIFF_DATAFRAME FROM TWO FILES
- MOVE/COPY&WATERMARK PDFS!
- CREATE .ZIP FILE OF WATERMARKED PDFS
- EMAIL AND SEND .ZIP FILE and .CSV FILES

---
### *DEFINE IMPORTS*

In [ ]:
# IMPORT THE GOODS
import os, sys, time
from time import sleep
from pathlib import Path
import fnmatch, glob, shutil
import pandas as pd
import numpy as np
from pandas import Series, DataFrame

In [ ]:
import tempfile, subprocess
from PyPDF2 import PdfFileReader, PdfFileWriter
from zipfile import ZipFile

In [ ]:
import email, smtplib, ssl
import os.path as op
from email import encoders
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.utils import COMMASPACE, formatdate

### *DEFINE FUNCTIONS*

In [ ]:
"""
TAKES IN:
(1) path to pdf 
RETURNS: 
the STRING of that PDF to match the naming convention
in G:/C of A's/#Email Node/ from F:/APPS/CofA/
"""
def generate_naming_convention(the_pdf_path): #{
    # get/set filename to variable
    the_file_name = str(os.path.basename(the_pdf_path))
    # PERFORM STRING OPERATIONS
    #################################
    idx_mrk = the_file_name.rfind('@', 0, len(the_file_name))
    half1 = str(the_file_name[0:idx_mrk])
    half2 = str(the_file_name[idx_mrk + 1:len(the_file_name)])
    print("\t\t[*************************]")
    print("\t\t|>>> HALF 1 == " + half1)
    print("\t\t|>>> HALF 2 == " + half2)
    #  setup NEW FILE NAME (for copy)
    new_name = "part "
    new_name += str(half1)
    new_name += " CofA Lot# "
    new_name += str(half2)
    print("\t\t|>>> NEW NAME == " + str(new_name))
    print("\t\t[*************************]")
    #################################
    return str(new_name)
#}

In [ ]:
"""
TAKES IN:
PDF TO WATERMARK,
"""
def create_watermark(input_pdf, output, watermark): # {
    try:  # {
        watermark_obj = PdfFileReader(watermark)
        watermark_page = watermark_obj.getPage(0)

        pdf_reader = PdfFileReader(input_pdf)
        pdf_writer = PdfFileWriter()

        # Watermark all the pages
        for page in range(pdf_reader.getNumPages()):  # {
            page = pdf_reader.getPage(page)
            page.mergePage(watermark_page)
            pdf_writer.addPage(page)
        # }

        with open(output, 'wb') as out:  # {
            pdf_writer.write(out)
        # }
    # }
    except: # {
        errorMessage = str(sys.exc_info()[0]) + "\n\t\t"
        errorMessage = errorMessage + str(sys.exc_info()[1]) + "\n\t\t"
        errorMessage = errorMessage + str(sys.exc_info()[2]) + "\n"
        exc_type, exc_obj, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        typeE = str("TYPE : " + str(exc_type))
        fileE = str("FILE : " + str(fname))
        lineE = str("LINE : " + str(exc_tb.tb_lineno))
        messageE = str("MESG : " + "\n" + str(errorMessage) + "\n")
        logging.error("\n" + typeE +
              "\n" + fileE +
              "\n" + lineE +
              "\n" + messageE)
        # }
    else: # {
        print("\t\t[watermark-pdf] FIN...")
    # }
    finally: # {
        # CREATE END-TIME VAR
        time_end = pd.Timestamp.now()
        # DETERMINE OVERALL RUN-TIME
        run_time = pd.Timedelta(time_end - time_start)
        # PRINT TOTAL RUNTIME
        print("\t\t[watermark-pdf] >>> time_alloted: " + str(run_time))
    # }
    return
# }

In [ ]:
def get_all_file_paths(directory): #{
    
    # initializing empty file paths list 
    file_paths = [] 
    
    # crawling through directory and subdirectories 
    for root, directories, files in os.walk(directory): #{
        for filename in files: #{
            # join the two strings in order to form the full filepath. 
            filepath = os.path.join(root, filename) 
            file_paths.append(filepath)
        #}
    #}
    
    # returning all file paths
    return file_paths
#}

In [1]:
def send_mail(send_from, send_to, subject, message, files=[], 
              server="cos.smtp.agilent.com", port=587, use_tls=True): #{
    print("SENDING MAIL... DATE == " + str(pd.Timestamp.now())[:10])
    msg = MIMEMultipart()
    msg['From'] = send_from
    msg['To'] = COMMASPACE.join(send_to)
    msg['Date'] = formatdate(localtime=True)
    msg['Subject'] = subject
    
    msg.attach(MIMEText(message))
    
    for path in files: #{
        part = MIMEBase('application', "octet-stream")
        with open(path, 'rb') as file: #{
            part.set_payload(file.read())
        #}
        encoders.encode_base64(part)
        part.add_header('Content-Disposition',
                       'attachment; filename="{}"'.format(op.basename(path)))
        msg.attach(part)
    #}
    
    smtp = smtplib.SMTP(server, port)
    if use_tls: #{
        smtp.starttls()
    #}
    smtp.sendmail(send_from, send_to, msg.as_string())
    smtp.quit()
#}

In [ ]:
def main(): #{
    
#}

---
## `__boilterplate__`

In [ ]:
if __name__ == "__main__": #{
    # SETUP LOGGER
    ##############################
    # INSTANTIATE GLOBAL VARIABLES
    in_file = "C:/CofA/imp/Agilent_CofA_Letterhead_03-21-19.pdf"
    save_list = []
    in_directory = "C:/Temp/F/APPS/CofA/"  #"F:/APPS/CofA/"
    out_directory = "C:/Temp/G/C of A's/#Email Node/"  #"G:/C of A's/#Email Node/"
    to_email = "agilent_cofa@agilent.com"
    from_email = "derek.bates@non.agilent.com"
    today_date_str = "2019-08-14"
#}